# End-to-End Optical Design with 5 Lines of Code

This notebook demonstrates end-to-end optimization of optical systems using DeepLens. The process involves only 5 key lines of code for the core optimization loop.

Reference Paper:
Xinge Yang, Qiang Fu and Wolfgang Heidrich, "Curriculum learning for ab initio deep learned refractive optics," Nature Communications 2024.

In [ ]:
# Import required libraries
import logging
import os
import random
import string
from datetime import datetime

import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import wandb
import yaml
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from deeplens import GeoLens
from deeplens.network import UNet, NAFNet
from deeplens.network.dataset import ImageDataset
from deeplens.utils import (
    batch_PSNR,
    batch_SSIM,
    denormalize_ImageNet,
    normalize_ImageNet,
    set_logger,
    set_seed,
)

# Set plotting style
plt.style.use('seaborn')
%matplotlib inline

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Configuration

Let's set up the experiment configuration for end-to-end optimization:

In [ ]:
def config():
    # Load config
    with open("configs/1_end2end_5lines.yml") as f:
        args = yaml.load(f, Loader=yaml.FullLoader)

    # Create result directory
    characters = string.ascii_letters + string.digits
    random_string = "".join(random.choice(characters) for i in range(4))
    current_time = datetime.now().strftime("%m%d-%H%M%S")
    exp_name = current_time + "-End2End-5-lines-" + random_string
    result_dir = f"./results/{exp_name}"
    os.makedirs(result_dir, exist_ok=True)
    args["result_dir"] = result_dir

    # Set random seed
    if args["seed"] is None:
        seed = random.randint(0, 100)
        args["seed"] = seed
    set_seed(args["seed"])

    # Log
    set_logger(result_dir)
    logging.info(f'EXP: {args["EXP_NAME"]}')
    
    # Device info
    num_gpus = torch.cuda.device_count()
    args["num_gpus"] = num_gpus
    args["device"] = device
    logging.info(f"Using {num_gpus} GPU(s)")

    # Save config
    with open(f"{result_dir}/config.yml", "w") as f:
        yaml.dump(args, f)
        
    return args

# Load configuration
args = config()
print(f"Result directory: {args['result_dir']}")
print(f"Random seed: {args['seed']}")

## Line 1: Load the Lens

First, we load our initial lens design and set up the neural network:

In [ ]:
# ========================================
# Line 1: load a lens
# ========================================
lens = GeoLens(filename=args["lens"]["path"])
lens.change_sensor_res(args["train"]["img_res"])

# Initialize the neural network
net = NAFNet(
    in_chan=3, 
    out_chan=3, 
    width=16, 
    middle_blk_num=1, 
    enc_blk_nums=[1, 1, 1, 18], 
    dec_blk_nums=[1, 1, 1, 1]
)
net = net.to(lens.device)

# Load pretrained weights if specified
if args["network"]["pretrained"]:
    net.load_state_dict(torch.load(args["network"]["pretrained"]))

# Print lens information
print(f"Lens focal length: {lens.foclen:.2f} mm")
print(f"Lens f-number: f/{lens.fnum:.2f}")
print(f"Lens field of view: {lens.hfov*2*57.3:.2f} degrees")
print(f"Sensor size: {lens.sensor_size[0]:.2f} x {lens.sensor_size[1]:.2f} mm")